In [2]:
import pandas as pd
import re
import kss
from tqdm import tqdm
from ekonlpy.sentiment import MPCK
tqdm.pandas()

# 1. 파일 경로 설정 (윈도우 경로는 앞에 r을 붙여주는 것이 좋아요!)
file_path = r'C:\KDT\bok_pjt\team2_forecast_pjt\db\preprocessing\prepro delete under 15_with_sent_id.parquet'

# 2. 데이터 불러오기 
# 메모리를 아끼기 위해 필요한 기둥(doc_id, category)만 가져올게요.
print("🔍 통합 파일을 불러오고 있습니다...")
df_total = pd.read_parquet(file_path, columns=['doc_id', 'category'])

# 3. '의사록' 카테고리만 필터링하기
# 만약 파일 안의 명칭이 '의사록'이 아니라 'meeting' 등으로 되어 있을 수 있으니 확인이 필요해요!
meeting_ids = df_total[df_total['category'] == '의사록']['doc_id'].unique()

# 4. 결과 확인
print(f"✅ 추출 완료! 총 {len(meeting_ids)}개의 의사록 문서 번호를 찾았습니다.")
print(f"🆔 추출된 번호 예시: {meeting_ids[:10]}")

# --- [팁: 나중에 원본 데이터와 합칠 때] ---
# 이제 이 meeting_ids를 리스트나 시리즈 형태로 가지고 있다가 
# 다시 작업할 의사록 원본 데이터의 'doc_id' 컬럼에 쏙 넣어주면 됩니다!

🔍 통합 파일을 불러오고 있습니다...
✅ 추출 완료! 총 150개의 의사록 문서 번호를 찾았습니다.
🆔 추출된 번호 예시: [239456 239457 239458 239459 239460 239461 239462 239463 239464 239465]


In [3]:
# 2. 원본 의사록 CSV 로드 및 doc_id 부여
print("📂 원본 의사록 데이터를 로드합니다...")
meetings = pd.read_csv('meeting_preprocessed_fixed.csv', encoding='utf-8')

# 개수가 맞는지 확인 후 doc_id 삽입
if len(meetings) == len(meeting_ids):
    meetings['doc_id'] = meeting_ids
    print("✅ doc_id 매칭 성공!")
else:
    print(f"⚠️ 경고: 개수가 다릅니다! (CSV: {len(meetings)}개, Parquet: {len(meeting_ids)}개)")


# 전처리 적용


# 4. 문장 분리 (KSS) 및 Explode
print("✂️ 문장 분리(KSS) 시작...")
meetings['content'] = meetings['content'].progress_apply(kss.split_sentences)
df_sentences = meetings.explode('content').reset_index(drop=True)

📂 원본 의사록 데이터를 로드합니다...
✅ doc_id 매칭 성공!
✂️ 문장 분리(KSS) 시작...


  0%|          | 0/150 [00:00<?, ?it/s][Kss]: Oh! You have konlpy.tag.Mecab in your environment. Kss will take this as a backend! :D

100%|██████████| 150/150 [00:41<00:00,  3.62it/s]


In [4]:
# 5. 토큰화 및 n-gram (학생분의 기존 함수 활용)
def ngramize(tokens, max_n=5):
    keep_tags = ['NNG', 'VA', 'VAX', 'MAG', 'VV']
    filtered = [w for w in tokens if w.split('/')[1] in keep_tags]
    all_ngrams = []
    for pos in range(len(filtered)):
        for n in range(1, max_n + 1):
            if pos + n <= len(filtered):
                ngram = ";".join(filtered[pos : pos + n])
                all_ngrams.append({'ngram': ngram, 'start': pos, 'end': pos + n, 'len': n})
                
    final_ngrams = []
    sorted_ngrams = sorted(all_ngrams, key=lambda x: x['len'], reverse=True)
    covered_ranges = set()
    for ngram in sorted_ngrams:
        is_covered = False
        for i in range(ngram['start'], ngram['end']):
            if i in covered_ranges:
                is_covered = True
                break
        if not is_covered:
            final_ngrams.append(ngram['ngram'])
            for i in range(ngram['start'], ngram['end']):
                covered_ranges.add(i)
                
    return final_ngrams

from ekonlpy.sentiment import MPCK
mpck = MPCK()
def get_final_tokens(text):
    basic_tokens = mpck.tokenize(text)

    return ngramize(basic_tokens, max_n=5)

print("🔤 토큰화 및 n-gram 생성 중...")
df_sentences['tokens'] = df_sentences['content'].progress_apply(get_final_tokens)

# 6. 최종 확인 및 저장
print("✨ 작업 완료!")
print(df_sentences[['doc_id', 'content', 'tokens']].head(10))

🔤 토큰화 및 n-gram 생성 중...


100%|██████████| 14342/14342 [00:19<00:00, 726.88it/s] 

✨ 작업 완료!
   doc_id                                            content  \
0  239456  최근 건설투자 등 일부 경제지표 가 지난해 12월 예상했던 것보다 부진한 것으로 나...   
1  239456  다른 소매판매 설비투자 등 내수지표의 하락세가 뚜렷하고 수출이 둔화되고 있으며 생산...   
2  239456  2013년부터 새로 시작되는 3년간의 중기 물가안정목표를 금년중 새 로 설정해야 하...   
3  239456  과거 선거가 있던 해에는 물가가 평소보다 높아지는 경향이 있었음을 감안할 때 금년도...   
4  239456  경상수지의 흑자 지속 풍부한 국제유동성의 유입 등으로 원 화가 기조적으로 강세를 나...   
5  239456  환율이 상승할 경우 물가상승효과가 뚜렷한 반면 환율이 하락할 경우에는 물가하락효과의...   
6  239456  이러한 고용구조의 변화가 고용의 양극화 소득의 재분배 악 화 등 고용시장의 불균형을...   
7  239456  한편 경제전망시에는 전망 작업 결과로 나타나는 수치보다는 균 형된 시각과 일관성을 ...   
8  239456  최근 국내은행의 외화자금사정 이 크게 악화되지 않을 것으로 보이지만 국내은행의 단기...   
9  239456  우리의 주요 무역상대국인 중국과의 수출입 거래시 원화와 위안화를 이용해서 수취 내지...   

                                              tokens  
0  [건설/NNG;투자/NNG;경제/NNG;지표/NNG;예상/NNG, 하/VV;부진/N...  
1  [소매/NNG;판매/NNG;설비/NNG;투자/NNG;내수/NNG, 지표/NNG;하락...  
2  [새로/MAG;시작/NNG;중기/NNG;물가/NNG;안정/NNG, 목표/NNG;중/...  
3  [선거/NNG;있/VA;물가/NNG;평소/NNG;높/VA, 경향/NNG;있/VA;감...  
4  [경상/NNG;

In [5]:
df_sentences['sent_id'] = df_sentences.groupby('doc_id').cumcount() + 1

# 확인
print(df_sentences[['doc_id', 'sent_id', 'content', 'tokens']].head())

   doc_id  sent_id                                            content  \
0  239456        1  최근 건설투자 등 일부 경제지표 가 지난해 12월 예상했던 것보다 부진한 것으로 나...   
1  239456        2  다른 소매판매 설비투자 등 내수지표의 하락세가 뚜렷하고 수출이 둔화되고 있으며 생산...   
2  239456        3  2013년부터 새로 시작되는 3년간의 중기 물가안정목표를 금년중 새 로 설정해야 하...   
3  239456        4  과거 선거가 있던 해에는 물가가 평소보다 높아지는 경향이 있었음을 감안할 때 금년도...   
4  239456        5  경상수지의 흑자 지속 풍부한 국제유동성의 유입 등으로 원 화가 기조적으로 강세를 나...   

                                              tokens  
0  [건설/NNG;투자/NNG;경제/NNG;지표/NNG;예상/NNG, 하/VV;부진/N...  
1  [소매/NNG;판매/NNG;설비/NNG;투자/NNG;내수/NNG, 지표/NNG;하락...  
2  [새로/MAG;시작/NNG;중기/NNG;물가/NNG;안정/NNG, 목표/NNG;중/...  
3  [선거/NNG;있/VA;물가/NNG;평소/NNG;높/VA, 경향/NNG;있/VA;감...  
4  [경상/NNG;수지/NNG;흑자/NNG;지속/NNG;풍부/NNG, 국제/NNG;유동...  


In [7]:
from collections import Counter
# 1. 기존의 전체 통합 파일을 불러옵니다.
df_total = pd.read_parquet(file_path)

# 2. 전체 데이터에서 '오류가 있던 기존 의사록'만 삭제합니다.
# 우리가 아까 뽑아놓은 meeting_ids(의사록 번호표들)를 사용해요.
# ~ 기호는 "이 번호가 아닌 것들만 남겨라"라는 뜻이에요.
df_total_no_meeting = df_total[~df_total['doc_id'].isin(meeting_ids)].copy()

# 3. '새로 작업한 깨끗한 의사록(df_sentences)'을 합쳐줍니다.
# 이제 바구니에 뉴스가 담긴 부분 + 새로 고친 의사록 부분이 합쳐집니다.
df_final_total = pd.concat([df_total_no_meeting, df_sentences], ignore_index=True)

print("📊 전체 단어 빈도수를 다시 계산합니다...")
all_tokens = [t for sublist in df_final_total['tokens'] for t in sublist]
token_counts = Counter(all_tokens)
valid_tokens = {token for token, count in token_counts.items() if count > 15}

meeting_mask = df_final_total['category'] == '의사록'

# 의사록 행의 'tokens' 컬럼에만 필터 함수를 적용해요.
df_final_total.loc[meeting_mask, 'tokens'] = df_final_total.loc[meeting_mask, 'tokens'].apply(
    lambda x: [t for t in x if t in valid_tokens]
)

# 4. (선택사항) 보기 좋게 doc_id 순서대로 다시 정렬해줄게요.
df_final_total = df_final_total.sort_values(by=['doc_id']).reset_index(drop=True)

df_final_total['tokens'] = df_final_total['tokens'].apply(str)

# 5. 이제 진짜 '업데이트된 전체 통합 파일'을 저장합니다!
df_final_total.to_parquet(r'C:\KDT\bok_pjt\team2_forecast_pjt\db\preprocessing\prepro_final_v3.parquet')

print("✨ 바꿔치기 완료! 이제 전체 파일에 고쳐진 의사록이 들어있습니다.")

📊 전체 단어 빈도수를 다시 계산합니다...
✨ 바꿔치기 완료! 이제 전체 파일에 고쳐진 의사록이 들어있습니다.


In [21]:
# 1. doc_id를 기준으로 나타나는 순서대로 새 번호를 매깁니다.
# 똑같은 원래 doc_id는 똑같은 새 번호를 받게 됩니다.
df_final_total['doc_id'] = pd.factorize(df_final_total['doc_id'])[0]

# 2. 잘 되었는지 확인하기 (상위 10개 출력)
# 같은 doc_id가 유지되는지 눈으로 확인해 보세요!
print(df_final_total.head(10))

   doc_id  sent_id        date  \
0       0       12  2025-11-14   
1       0       13  2025-11-14   
2       0        7  2025-11-14   
3       0        6  2025-11-14   
4       0       11  2025-11-14   
5       0       10  2025-11-14   
6       0        5  2025-11-14   
7       0        4  2025-11-14   
8       0        2  2025-11-14   
9       0        3  2025-11-14   

                                             content  \
0                     반면 개인 투자자는 홀로 3조2336억원을 순매수했다.   
1  업종별로는 전기 전자 6.21 보험 5.35 기계 장비 4.82 의료 정밀기기 4....   
2  13일 미국 동부시간 뉴욕증권거래소 NYSE 에서 스탠더드앤드푸어스 S P 500 ...   
3  간밤 뉴욕증시가 인공지능 AI 산업 고평가 우려와 미 연방준비제도 Fed 연준 인사...   
4   유가증권시장에서 외국인과 기관 투자자는 각각 2조3667억원 9002억원을 순매도했다.   
5  이경민 대신증권 연구원은 금리 인하 기대감이 후퇴하면서 성장 기술주 섹터의 부담이 ...   
6  이날 지수는 전일 대비 108.72포인트 2.61 급락한 4061.91에 출발한 뒤...   
7  14일 코스피는 전일 대비 159.06포인트 3.81 내린 4011.57에 거래를 ...   
8           나흘 연속 상승했던 코스피가 3 대 급락하며 4000선을 간신히 지켰다.   
9  간밤 미국 셧다운 해제에도 불구하고 금리인하 기대감이 약화한 데다 고평가 논란이 불...   

        

In [22]:
actual_ids = set(df_final_total['doc_id'])

# 2. 가장 작은 번호부터 가장 큰 번호까지 '완벽한 번호 세트'를 만듭니다.
start = min(actual_ids)
end = max(actual_ids)
perfect_ids = set(range(start, end + 1))

# 3. 완벽한 세트에서 내 데이터를 빼면? (빠진 번호만 남아요!)
missing_ids = perfect_ids - actual_ids

# 4. 결과 출력
if not missing_ids:
    print("✅ 오! 빠진 번호가 하나도 없네요. 완벽합니다!")
else:
    print(f"⚠️ 빠진 번호가 있어요! 총 {len(missing_ids)}개가 비어있습니다.")
    print(f"빠진 번호 목록: {sorted(list(missing_ids))} ... (앞부분 10개만 출력)")

✅ 오! 빠진 번호가 하나도 없네요. 완벽합니다!


In [ ]:
if isinstance(df_final_total['tokens'].iloc[0], list):
    df_final_total['tokens'] = df_final_total['tokens'].apply(str)
    print("📦 tokens 컬럼을 문자열로 변환했습니다. (저장 준비 완료)")

# 3. 최종 경로 설정 및 저장
# 버전 관리 차원에서 v4_perfect라고 이름을 붙여볼게요.
output_path = r'C:\KDT\bok_pjt\team2_forecast_pjt\db\preprocessing\preprocessed_260106.parquet'

try:
    df_final_total.to_parquet(output_path, index=False)
    print(f"✨ 축하합니다! 파일이 성공적으로 저장되었습니다.")
    print(f"📍 저장 위치: {output_path}")
except Exception as e:
    print(f"⚠️ 저장 중 에러가 발생했어요: {e}")

# 4. 잘 저장됐나 살짝 열어보기 (확인용)
test_df = pd.read_parquet(output_path)
print(f"\n✅ 저장된 데이터 개수: {len(test_df)}개")


✨ 축하합니다! 파일이 성공적으로 저장되었습니다.
📍 저장 위치: C:\KDT\bok_pjt\team2_forecast_pjt\db\preprocessing\preprocessed_260106.parquet

✅ 저장된 데이터 개수: 4974767개
   doc_id                                            content  \
0       0                     반면 개인 투자자는 홀로 3조2336억원을 순매수했다.   
1       0  업종별로는 전기 전자 6.21 보험 5.35 기계 장비 4.82 의료 정밀기기 4....   
2       0  13일 미국 동부시간 뉴욕증권거래소 NYSE 에서 스탠더드앤드푸어스 S P 500 ...   
3       0  간밤 뉴욕증시가 인공지능 AI 산업 고평가 우려와 미 연방준비제도 Fed 연준 인사...   
4       0   유가증권시장에서 외국인과 기관 투자자는 각각 2조3667억원 9002억원을 순매도했다.   

                                  tokens  
0  ["반면/NNG;개인/NNG;홀로/MAG;순매수/NNG;하/VV"]  
1                                     []  
2    ["포인트/NNG;내리/VV;장/NNG;마감/NNG;하/VV"]  
3                                     []  
4        ["외국인/NNG;기관/NNG;순매도/NNG;하/VV"]  


In [24]:
test_df.head()

,doc_id,sent_id,date,content,tokens,category,source
0,0,12,2025-11-14,반면 개인 투자자는 홀로 3조2336억원을 순매수했다.,"[""반면/NNG;개인/NNG;홀로/MAG;순매수/NNG;하/VV""]",뉴스,매일경제
1,0,13,2025-11-14,업종별로는 전기 전자 6.21 보험 5.35 기계 장비 4.82 의료 정밀기기 4....,[],뉴스,매일경제
2,0,7,2025-11-14,13일 미국 동부시간 뉴욕증권거래소 NYSE 에서 스탠더드앤드푸어스 S P 500 ...,"[""포인트/NNG;내리/VV;장/NNG;마감/NNG;하/VV""]",뉴스,매일경제
3,0,6,2025-11-14,간밤 뉴욕증시가 인공지능 AI 산업 고평가 우려와 미 연방준비제도 Fed 연준 인사...,[],뉴스,매일경제
4,0,11,2025-11-14,유가증권시장에서 외국인과 기관 투자자는 각각 2조3667억원 9002억원을 순매도했다.,"[""외국인/NNG;기관/NNG;순매도/NNG;하/VV""]",뉴스,매일경제
